In [7]:
import os
from tqdm import tqdm
import subprocess

In [8]:
# Task 1: create msh files by ./runme.sh 0 30 10 200

In [9]:
# Task2: convert msh to xml by dolfin-convert <msh> <xml>

In [23]:
subprocess.run(["ls", "../murtazo/cloudnaca/msh/"], check=True, text=True, capture_output=True)

CompletedProcess(args=['ls', '../murtazo/cloudnaca/msh/'], returncode=0, stdout='r0a0n200.msh\nr0a12n200.msh\nr0a15n200.msh\nr0a18n200.msh\nr0a21n200.msh\nr0a24n200.msh\nr0a27n200.msh\nr0a30n200.msh\nr0a3n200.msh\nr0a6n200.msh\nr0a9n200.msh\nr1a0n200.msh\nr1a12n200.msh\nr1a15n200.msh\nr1a18n200.msh\nr1a21n200.msh\nr1a24n200.msh\nr1a27n200.msh\nr1a30n200.msh\nr1a3n200.msh\nr1a6n200.msh\nr1a9n200.msh\nr2a0n200.msh\nr2a12n200.msh\nr2a15n200.msh\nr2a15n200.xml\nr2a18n200.msh\nr2a21n200.msh\nr2a24n200.msh\nr2a27n200.msh\nr2a30n200.msh\nr2a3n200.msh\nr2a6n200.msh\nr2a9n200.msh\nr3a0n200.msh\nr3a12n200.msh\nr3a15n200.msh\nr3a18n200.msh\nr3a21n200.msh\nr3a24n200.msh\nr3a27n200.msh\nr3a30n200.msh\nr3a3n200.msh\nr3a6n200.msh\nr3a9n200.msh\n', stderr='')

In [4]:
input_path = "../murtazo/cloudnaca/msh"
for filename in tqdm(os.listdir(input_path)):
    print(filename)
    #Task 1 convert msh to xml by dolfin-convert <msh> <xml>
    subprocess.run(['dolfin-convert', filename, r2a15n200.xml])
    #Task 2 run navier_stokes_solver by ./airfoil 10 0.0001 10. 1 file
    #Task3 getResults by Tail –n 1 drag_ligt.m

    
    
 

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 64439.63it/s]

r0a24n200.msh
r0a15n200.msh
r0a3n200.msh
r0a6n200.msh
r2a12n200.msh
r1a27n200.msh
r1a3n200.msh
r1a30n200.msh
r0a0n200.msh
r2a6n200.msh
r2a21n200.msh
r2a15n200.xml
r2a15n200.msh
r3a18n200.msh
r3a15n200.msh
r1a18n200.msh
r3a12n200.msh
r3a6n200.msh
r3a30n200.msh
r1a6n200.msh
r1a12n200.msh
r2a3n200.msh
r1a21n200.msh
r0a30n200.msh
r0a21n200.msh
r2a24n200.msh
r1a0n200.msh
r2a18n200.msh
r0a27n200.msh
r2a27n200.msh
r1a9n200.msh
r3a9n200.msh
r3a27n200.msh
r1a24n200.msh
r3a21n200.msh
r1a15n200.msh
r2a9n200.msh
r0a12n200.msh
r3a0n200.msh
r0a18n200.msh
r3a3n200.msh
r2a30n200.msh
r3a24n200.msh
r2a0n200.msh
r0a9n200.msh


In [ ]:
# Task 3 run navier_stokes_solver by ./airfoil 10 0.0001 10. 1 file